In [1]:
# Create a virtual environment named 'myenv' with Python 3.10
!python3.10 -m venv myenv

# Activate the environment (Linux/macOS style)
!source myenv/bin/activate

# Upgrade pip inside the venv
!pip install --upgrade pip

# Install required libraries (CPU-only TensorFlow setup)
!pip install numpy opencv-python tensorflow

Error: Venv creation is not allowed. A Studio has a default conda environment (max 1 environment). Start a new Studio to create a new environment.
zsh:source:1: no such file or directory: myenv/bin/activate
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 306.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 220.6 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 192.6 MB/s  0:00:020:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 313.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 208.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 225.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 236.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 327.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 173.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 266.1 MB/s  0:00:00
   ━━━━━━

In [3]:
import os, glob, random, cv2
import numpy as np
import tensorflow as tf
!pip install kagglehub
import kagglehub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [kagglehub]/3 [kagglesdk]


In [4]:
# ===== Cell 1: Setup & Dataset =====
import os, glob, random, cv2
import numpy as np
import tensorflow as tf
import kagglehub

# Download dataset (cached after first run)
dataset_path = kagglehub.dataset_download(
    "jedidiahangekouakou/grid-corpus-dataset-for-training-lipnet"
)
print("Dataset path:", dataset_path)

# Vocabulary
vocab = list("abcdefghijklmnopqrstuvwxyz'?!123456789 ")
char_to_num = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="")
num_to_char = tf.keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(),
    invert=True,
    oov_token=""
)


100%|██████████| 12.6G/12.6G [02:21<00:00, 95.8MB/s]

Extracting files...


Dataset path: /teamspace/studios/this_studio/.cache/kagglehub/datasets/jedidiahangekouakou/grid-corpus-dataset-for-training-lipnet/versions/1


I0000 00:00:1769011327.389894   31041 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78765 MB memory:  -> device: 0, name: NVIDIA H100 80GB HBM3, pci bus id: 0000:06:00.0, compute capability: 9.0


In [8]:
import os, glob

print("Listing dataset_path:")
print(os.listdir(dataset_path))


Listing dataset_path:
['data']


In [9]:
# ===== Cell 2: Data loading & pipeline =====

def load_video(path):
    cap = cv2.VideoCapture(path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = frame[190:236, 80:220]
        frame = np.expand_dims(frame, -1)
        frames.append(frame)
        if len(frames) == 75:
            break
    cap.release()

    if len(frames) < 75:
        return np.zeros((75, 46, 140, 1), np.float32)

    frames = np.array(frames, np.float32)
    return (frames - frames.mean()) / (frames.std() + 1e-6)


def load_alignments(path):
    chars = []
    with open(path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if parts[2] != "sil":
                for c in " " + parts[2]:
                    chars.append(c)
    ids = char_to_num(tf.constant(chars)).numpy().astype(np.int32)
    return ids[1:]


def load_data(path):
    path = path.numpy().decode()
    name = os.path.splitext(os.path.basename(path))[0]
    align_path = os.path.join(os.path.dirname(path), "align", name + ".align")
    return load_video(path), load_alignments(align_path)


def map_fn(path):
    x, y = tf.py_function(load_data, [path], [tf.float32, tf.int32])
    x.set_shape([75, 46, 140, 1])
    y.set_shape([None])
    return x, y


# Collect files
video_files = sorted(
    glob.glob(os.path.join(dataset_path, "data","s*_processed", "*.mpg"))
)
random.shuffle(video_files)

split = int(0.9 * len(video_files))
train_files = video_files[:split]
test_files  = video_files[split:]

print("Train:", len(train_files), "Test:", len(test_files))

BATCH = 2

train = tf.data.Dataset.from_tensor_slices(train_files)
train = train.map(map_fn, num_parallel_calls=tf.data.AUTOTUNE)
train = train.padded_batch(
    BATCH,
    padded_shapes=([75,46,140,1],[None]),
    padding_values=(0.0, 0),
    drop_remainder=True
)
train = train.repeat()
train = train.prefetch(tf.data.AUTOTUNE)

test = tf.data.Dataset.from_tensor_slices(test_files)
test = test.map(map_fn, num_parallel_calls=tf.data.AUTOTUNE)
test = test.padded_batch(
    BATCH,
    padded_shapes=([75,46,140,1],[None]),
    padding_values=(0.0, 0)
)
test = test.prefetch(tf.data.AUTOTUNE)


Train: 29700 Test: 3300


In [ ]:
# ===== Cell 3: Model, loss, callbacks, training =====
from tensorflow.keras.layers import (
    Conv3D, MaxPool3D, Bidirectional, LSTM,
    Dense, Dropout, Reshape, Input
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# Model
model = Sequential([
    Input(shape=(75,46,140,1)),
    Conv3D(128, 3, padding="same", activation="relu"),
    MaxPool3D((1,2,2)),
    Conv3D(256, 3, padding="same", activation="relu"),
    MaxPool3D((1,2,2)),
    Conv3D(75, 3, padding="same", activation="relu"),
    MaxPool3D((1,2,2)),
    Reshape((75, 5*17*75)),
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.5),
    Dense(len(char_to_num.get_vocabulary()) + 1, activation="softmax")
])

def CTCLoss(y_true, y_pred):
    batch = tf.shape(y_true)[0]
    input_len = tf.fill([batch,1], tf.shape(y_pred)[1])
    label_len = tf.math.count_nonzero(y_true, axis=1, keepdims=True)
    return tf.keras.backend.ctc_batch_cost(
        y_true, y_pred, input_len, label_len
    )

model.compile(optimizer=Adam(1e-4), loss=CTCLoss)

os.makedirs("models", exist_ok=True)
checkpoint = ModelCheckpoint(
    "models/epoch_{epoch:02d}.weights.h5",
    save_weights_only=True,
    verbose=1
)

class ProduceExample(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        x, y = next(iter(test.take(1)))
        yhat = self.model.predict(x, verbose=0)
        decoded, _ = tf.keras.backend.ctc_decode(
            yhat,
            input_length=[yhat.shape[1]] * yhat.shape[0],
            greedy=True
        )
        print(f"\nEpoch {epoch+1} samples:")
        for i in range(min(2, x.shape[0])):
            gt = tf.strings.reduce_join(num_to_char(y[i])).numpy().decode()
            pr = tf.strings.reduce_join(num_to_char(decoded[0][i])).numpy().decode()
            print("GT :", gt)
            print("PR :", pr)
            print("-"*50)

STEPS = len(train_files) // BATCH

model.fit(
    train,
    validation_data=test,
    epochs=50,
    steps_per_epoch=STEPS,
    callbacks=[checkpoint, ProduceExample()]
)


Epoch 1/50


2026-01-21 16:07:36.390762: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 90501


 1709/14850 ━━━━━━━━━━━━━━━━━━━━ 6:52 31ms/step - loss: 72.4584

[mpeg1video @ 0x76f0f8005ac0] ac-tex damaged at 22 17
[mpeg1video @ 0x76f0f8005ac0] Warning MVs not available


 1779/14850 ━━━━━━━━━━━━━━━━━━━━ 6:50 31ms/step - loss: 72.2150

...


Epoch 8: saving model to models/epoch_08.weights.h5

Epoch 8: finished saving model to models/epoch_08.weights.h5

Epoch 8 samples:
GT : bin red by k two please
PR : bin red by k six please
--------------------------------------------------
GT : place red at g seven now
PR : place red in c seven now
--------------------------------------------------
14850/14850 ━━━━━━━━━━━━━━━━━━━━ 497s 33ms/step - loss: 8.6086 - val_loss: 9.0138
Epoch 9/50
 1709/14850 ━━━━━━━━━━━━━━━━━━━━ 6:52 31ms/step - loss: 8.0910

[mpeg1video @ 0x76f0e807cd80] ac-tex damaged at 22 17
[mpeg1video @ 0x76f0e807cd80] Warning MVs not available


 1779/14850 ━━━━━━━━━━━━━━━━━━━━ 6:49 31ms/step - loss: 8.0893

[mpeg1video @ 0x76f0cc00de00] ac-tex damaged at 22 17
[mpeg1video @ 0x76f0cc00de00] Warning MVs not available


 3389/14850 ━━━━━━━━━━━━━━━━━━━━ 6:00 31ms/step - loss: 8.0627